In [277]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import bernoulli
from scipy.stats import gamma as gamma_dist
from scipy.stats import norm
from scipy.signal import convolve
import random
from os import path as path
import time
from scipy.optimize import minimize


For each combination of parameters, the function returns an array of size $2\times2\times T$.

* The first dimension is target presence (0 or 1)
* The second is response (0 or 1)
* the third is RT (in units of time points).

Entries are probabilities. The sum of the array is set to 2: that is, probabilities are conditional on target presence/absence.

In [311]:
def get_model_predictions(true_thetas, believed_thetas, gamma, shape, scale, T):
    
    # true thetas:  [p(1|absent), p(1|present)]
    # believed thetas: participants beliefs about the true thetas
    # gamma: the temporal discount factor
    # T number of time points to simulate
    # shape and scale: parameters of the gamma distribution for non-decision times. 
    
    #1. FIND POLICY
    
    # V is value. We simulate T*2 time here so that we can use the first T to get the threshold.
    V=np.full([T*2+1, T*2+1],np.nan);

    # A is action
    A=np.full([T*2+1, T*2+1],np.nan);

    # LLR is LLR
    LLR=np.full([T*2+1, T*2+1],np.nan);
    
    # Backward induction!
    for t in range(T*2, -1, -1):
        states= np.array(range(t+1)); # the state is the number of 1's observed so far
        LLR_at_t= states*(np.log(believed_thetas[1])- np.log(believed_thetas[0])) + \
        (t-states)*(np.log(1-believed_thetas[1])- np.log(1-believed_thetas[0]));
        
        LLR[t,states]= LLR_at_t;
        
        #to avoid overflow errors later. 
        clipped_LLR_at_t = np.clip(LLR_at_t,-500,500)

        P_present= np.exp(clipped_LLR_at_t)/(1+np.exp(clipped_LLR_at_t));
        
        # the expected value if I make a decision now is the probability of being correct
        V_choose_now= np.maximum(P_present,1-P_present)
        
        # We start by assuming that the agent makes a decision at the last time point, and move backward from there.
        if t==T*2:
            V[t, states]= V_choose_now;
            # I code 'target-present' decisions as 1, 'target-absent' decisions as 0, and waiting as np.nan
            A[t,states]= np.where(LLR_at_t>0,1,0);
            
        else:
            # Calculate value if waiting
            # the probability of obtaining a 1, marginalized over target presence and absence
            prob1= P_present*believed_thetas[1]+ (1-P_present)*believed_thetas[0];
            # the expected value if I wait is gamma times the expected value in the next time point.
            V_wait= gamma*(prob1*V[t+1,states+1]+ (1-prob1)*V[t+1,states]);
            
            # the expected value is the maximum between the expected value if I wait or if I make a decision.
            V[t,states] = np.maximum(V_choose_now,V_wait)
            
            A[t,states] = np.where(V_choose_now>V_wait,1,np.nan) * np.where(LLR_at_t>0,1,0)
        
    # 2. RUN SIMULATION FORWARD
    model_predictions = np.full([2,2,T],0.0);

    # run target absence trials
    prob=np.full([T+1, T+1],0.0);
    prob[0,0]=1
    for t in np.arange(0,T):
        for state in np.arange(0,t+1):
            if prob[t,state]==0:
                continue
            else:
                if not np.isnan(A[t,state]):
                    decision= int(A[t,state])
                    model_predictions[0,decision,t] = prob[t,state]
                else:
                    prob[t+1,state] += prob[t,state]*(1-true_thetas[0])
                    prob[t+1,state+1] += prob[t,state]*true_thetas[0]
                    
    # run target presence trials
    prob=np.full([T+1, T+1],0.0);
    prob[0,0]=1
    for t in np.arange(0,T):
        for state in np.arange(0,t+1):
            if prob[t,state]==0:
                continue
            else:
                if not np.isnan(A[t,state]):
                    decision= int(A[t,state])
                    model_predictions[1,decision,t] = prob[t,state]
                else:
                    prob[t+1,state] += prob[t,state]*(1-true_thetas[1])
                    prob[t+1,state+1] += prob[t,state]*true_thetas[1]
   
    if model_predictions.sum()<1.95:
       raise ValueError("increase T, a non-negligible number of trials is beyond the scope of the simulation")
    
    # Convolve with gamma
    gamma_pdf = gamma_dist.pdf(np.linspace(0, 100, 100), a=shape, scale=scale)
    gamma_pdf /= sum(gamma_pdf)
    
    # Prepare to store or plot the convolved data
    convolved_model_predictions = np.zeros_like(model_predictions)

    # Iterate over each combination in the first two dimensions and perform convolution
    for present in range(2):
        for response in range(2):
            # Convolve the gamma distribution with the current 1D slice
            # Using 'same' mode to ensure the output size matches the input
            convolved_model_predictions[present, response, :] = \
            convolve(model_predictions[present, response, :], gamma_pdf)[:model_predictions.shape[2]]+0.00001

    
    convolved_model_predictions[0, :, :] /= np.sum(convolved_model_predictions[0, :, :])
    convolved_model_predictions[1, :, :] /= np.sum(convolved_model_predictions[1, :, :])
    
    return(convolved_model_predictions)

In [ ]:
def get_model_predictions_ndt(true_thetas, believed_thetas, gamma, ndt, T):
    
    # true thetas:  [p(1|absent), p(1|present)]
    # believed thetas: participants beliefs about the true thetas
    # gamma: the temporal discount factor
    # T number of time points to simulate
    # non-decision time: non-decision time, in time-points 
    
    #1. FIND POLICY
    
    # V is value. We simulate T*2 time here so that we can use the first T to get the threshold.
    V=np.full([T*2+1, T*2+1],np.nan);

    # A is action
    A=np.full([T*2+1, T*2+1],np.nan);

    # LLR is LLR
    LLR=np.full([T*2+1, T*2+1],np.nan);
    
    # Backward induction!
    for t in range(T*2, -1, -1):
        states= np.array(range(t+1)); # the state is the number of 1's observed so far
        LLR_at_t= states*(np.log(believed_thetas[1])- np.log(believed_thetas[0])) + \
        (t-states)*(np.log(1-believed_thetas[1])- np.log(1-believed_thetas[0]));
        
        LLR[t,states]= LLR_at_t;
        
        #to avoid overflow errors later. 
        clipped_LLR_at_t = np.clip(LLR_at_t,-500,500)

        P_present= np.exp(clipped_LLR_at_t)/(1+np.exp(clipped_LLR_at_t));
        
        # the expected value if I make a decision now is the probability of being correct
        V_choose_now= np.maximum(P_present,1-P_present)
        
        # We start by assuming that the agent makes a decision at the last time point, and move backward from there.
        if t==T*2:
            V[t, states]= V_choose_now;
            # I code 'target-present' decisions as 1, 'target-absent' decisions as 0, and waiting as np.nan
            A[t,states]= np.where(LLR_at_t>0,1,0);
            
        else:
            # Calculate value if waiting
            # the probability of obtaining a 1, marginalized over target presence and absence
            prob1= P_present*believed_thetas[1]+ (1-P_present)*believed_thetas[0];
            # the expected value if I wait is gamma times the expected value in the next time point.
            V_wait= gamma*(prob1*V[t+1,states+1]+ (1-prob1)*V[t+1,states]);
            
            # the expected value is the maximum between the expected value if I wait or if I make a decision.
            V[t,states] = np.maximum(V_choose_now,V_wait)
            
            A[t,states] = np.where(V_choose_now>V_wait,1,np.nan) * np.where(LLR_at_t>0,1,0)
        
    # 2. RUN SIMULATION FORWARD
    model_predictions = np.full([2,2,T+ndt],0.0);

    # run target absence trials
    prob=np.full([T+1, T+1],0.0);
    prob[0,0]=1
    for t in np.arange(0,T):
        for state in np.arange(0,t+1):
            if prob[t,state]==0:
                continue
            else:
                if not np.isnan(A[t,state]):
                    decision= int(A[t,state])
                    model_predictions[0,decision,t+ndt] = prob[t,state]
                else:
                    prob[t+1,state] += prob[t,state]*(1-true_thetas[0])
                    prob[t+1,state+1] += prob[t,state]*true_thetas[0]
                    
    # run target presence trials
    prob=np.full([T+1, T+1],0.0);
    prob[0,0]=1
    for t in np.arange(0,T):
        for state in np.arange(0,t+1):
            if prob[t,state]==0:
                continue
            else:
                if not np.isnan(A[t,state]):
                    decision= int(A[t,state])
                    model_predictions[1,decision,t+ndt] = prob[t,state]
                else:
                    prob[t+1,state] += prob[t,state]*(1-true_thetas[1])
                    prob[t+1,state+1] += prob[t,state]*true_thetas[1]
   
    if model_predictions.sum()<1.95:
       raise ValueError("increase T, a non-negligible number of trials is beyond the scope of the simulation")
    
    model_predictions=model_predictions[:,:,:T]
    
    # Convolve with normal
    normal_pdf = norm.pdf(np.linspace(-50, 50, 100), loc=0, scale=1)
    normal_pdf /= sum(normal_pdf)
    
    # Prepare to store or plot the convolved data
    convolved_model_predictions = np.zeros_like(model_predictions)

    # Iterate over each combination in the first two dimensions and perform convolution
    for present in range(2):
        for response in range(2):
            # Convolve the gamma distribution with the current 1D slice
            # Using 'same' mode to ensure the output size matches the input
            convolved_model_predictions[present, response, :] = \
            convolve(model_predictions[present, response, :], normal_pdf)[:model_predictions.shape[2]]+0.00001

    
    convolved_model_predictions[0, :, :] /= np.sum(convolved_model_predictions[0, :, :])
    convolved_model_predictions[1, :, :] /= np.sum(convolved_model_predictions[1, :, :])
    
    return(convolved_model_predictions)

We need to fomat participants data in a way that is similar to convolved_model_predictions, but using counts instead of probabilities.

In [331]:
T=1000

df = pd.read_csv("data/E2a.csv") 
df['present'] = df['present'].replace(-1, 0)
df['decision'] = (((df['correct'] == 1) & (df['present'] == 1)) | ((df['correct'] == 0) & (df['present'] == 0))).astype(int)
df['rt'] = (df['rt'] / 0.05).round().astype(int)

df_filtered = df[df['subj_id'] == 1]

# Ensure all possible combinations are represented in the DataFrame, even those with zero counts
categories = {
    'present': [0, 1],
    'decision': [0, 1],
    'rt': range(T)
}

# Create a MultiIndex from the Cartesian product of the categories
index = pd.MultiIndex.from_product(categories.values(), names=categories.keys())

# Group by the necessary columns and count occurrences
df_grouped = df_filtered.groupby(['present', 'decision', 'rt']).size().reindex(index, fill_value=0)

# Unstack the MultiIndex DataFrame to create a 3D structure (2x2xT)
df_unstacked = df_grouped.unstack(level=['present', 'decision'])

subj1_data = df_unstacked.values.reshape((2, 2, len(categories['rt'])))

In [307]:
T=400

def negative_likelihood(params, data, T):
# params are in the order:
# true_theta0, [0]
# true theta1; [1]
# believed theta0; [2]
# believed theta1; [3]
# gamma for temporal discounting; [4]
# shape for gamma function; [5]
# scale for gamma function; [6]
# lapse rate; [7]
    predictions = get_model_predictions(params[:2], params[2:4], params[4], params[5], params[6], T);
    log_p = np.log(predictions)
    
    return(-np.sum(data*log_p))

In [243]:
params=[1,2,3,4,5,6,7,8,9,10]
params[2:4]

[3, 4]

In [ ]:
T=1000
# negative_likelihood([0.1,0.3,0.05,0.25,0.999,2,1],subj1_data,T)
initial_params = [0.15,0.2,0.15,0.2,0.99,1,1]
result = minimize(negative_likelihood, initial_params, args=(subj1_data,T), \
                  bounds = [(0,1),(0,1),(0,1),(0,1),(0.5,0.999),(0.001,10), (0.001,10)], \
                  method='Nelder-Mead')


C:\Users\tanzor\AppData\Local\Temp/ipykernel_5428/2795680548.py:92: RuntimeWarning: invalid value encountered in divide
  gamma_pdf /= sum(gamma_pdf)


In [326]:
result

           message: `gtol` termination condition is satisfied.
           success: True
            status: 1
               fun: 9679.215668491654
                 x: [ 2.230e-01  2.630e-01  2.230e-01  2.630e-01  9.373e-01
                      1.526e+00  1.028e+00]
               nit: 15
              nfev: 40
              njev: 5
              nhev: 0
          cg_niter: 16
      cg_stop_cond: 0
              grad: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
                      0.000e+00  0.000e+00]
   lagrangian_grad: [-1.438e-09 -1.448e-09 -1.438e-09 -1.448e-09  5.299e-10
                     -3.813e-09 -4.503e-09]
            constr: [array([ 2.230e-01,  2.630e-01,  2.230e-01,  2.630e-01,
                            9.373e-01,  1.526e+00,  1.028e+00])]
               jac: [<7x7 sparse matrix of type '<class 'numpy.float64'>'
                    	with 7 stored elements in Compressed Sparse Row format>]
       constr_nfev: [0]
       constr_njev: [0]
       constr_nh